In [9]:
import requests
import pandas as pd

# Set up the endpoint and query parameters.
url = "https://api.linkedin.com/rest/memberSnapshotData"
params = {
    "q": "criteria",      # Required query parameter
    "domain": "CONNECTIONS"  # Querying the CONTACTS domain for imported contacts
}

# Set up the headers with your access token and version header.
headers = {
    "Authorization": "Bearer API KEY",
    "Content-Type": "application/json",
    "LinkedIn-Version": "202312"  # Ensure you use the correct API version
}

# Make the GET request to the Member Snapshot API.
response = requests.get(url, params=params, headers=headers)

# Process the response.
if response.status_code == 200:
    snapshot_data = response.json()

    # Assuming snapshot_data contains an 'elements' list with one element for the CONTACTS domain.
    if 'elements' in snapshot_data and len(snapshot_data['elements']) > 0:
        contacts_snapshot = snapshot_data['elements'][0].get('snapshotData', [])

        # Convert the contacts list into a Pandas DataFrame.
        df = pd.DataFrame(contacts_snapshot)
    else:
        print("No snapshot data found for the CONTACTS domain.")
else:
    print("Error fetching snapshot data:", response.status_code)
    print(response.text)

df

,Company,Position,First Name,Connected On,Last Name,URL,Email Address
0,4impact capital,Tech & VC Intern,Ruben,11 Mar 2025,Stauch,https://www.linkedin.com/in/ruben-stauch,
1,WENVEST Capital,Early Stage Investor,Dr. Tom,11 Mar 2025,Orben,https://www.linkedin.com/in/dr-tom-orben-688833b6,
2,iii - financements,Financial Analyst,Hamza,11 Mar 2025,Bono Miftah,https://www.linkedin.com/in/hamza-bono-miftah-...,
3,Studydrive,Member Of The Board Of Advisors,Sven,10 Mar 2025,Gasper,https://www.linkedin.com/in/sven-gasper-43416877,
4,Earlybird Venture Capital,Investment Team & EagleEye,Nikolaus,10 Mar 2025,Lutterotti,https://www.linkedin.com/in/nikolaus-lutterott...,
...,...,...,...,...,...,...,...
1513,German Airways,Charter Operations & Produkt Manager,Maximilian,04 Sep 2021,Rauch,https://www.linkedin.com/in/maximilian-rauch,
1514,CIA Ruhr - Cyber & Identity Advisors,Founding Partner / IAM Lead,Kevin,04 Sep 2021,Loncsarszky,https://www.linkedin.com/in/kevin-loncsarszky-...,
1515,Volksbank Schermbeck eG,"EDV/IT, Systemadministration",Ingo,04 Sep 2021,Hamich,https://www.linkedin.com/in/ingo-hamich-ab59a747,
1516,RWE,Consultant - RWE Consulting,Michael,04 Sep 2021,Kreienbaum,https://www.linkedin.com/in/michael-kreienbaum,
